In [1]:
import pandas as pd
import geojson
import openrouteservice
import time
import os
import folium
import json
from shapely.geometry import Point, Polygon
from pyproj import Geod

import os
from dotenv import load_dotenv
load_dotenv('.env')

True

In [2]:
# authentication OpenRouteService
secret = os.getenv("ORS_SECRET")
ors = openrouteservice.Client(key=secret)

In [3]:
# load CSV
poi = pd.read_csv('./dummy/poi.csv')

posters = pd.read_csv('./dummy/p.csv')


posters['ID'] = posters['ID'].apply(lambda x : 'P' + str(x))
posters

,ID,latitude,longitude
0,P22,51.278824,4.409519
1,P49,51.182657,4.372914
2,P65,51.185132,4.344203
3,P77,51.171207,4.364278
4,P78,51.171212,4.364441
...,...,...,...
106,P655,51.220294,4.362614
107,P361,51.196552,4.408381
108,P929,51.217813,4.378362
109,P681,51.221088,4.488871


In [4]:
# extract coords
poi_coords = poi[["poi_name", "longitude", 'latitude']].values.tolist()
poster_coords= posters[["ID", "longitude", 'latitude']].values.tolist()

poster_coords

[['P22', 4.409519, 51.278824],
 ['P49', 4.3729141, 51.182657],
 ['P65', 4.3442028, 51.185132],
 ['P77', 4.364278, 51.171207],
 ['P78', 4.3644414, 51.171212],
 ['P80', 4.362545638, 51.16932],
 ['P84', 4.372675, 51.183229],
 ['P122', 4.453698, 51.238631],
 ['P132', 4.449170522, 51.251168],
 ['P137', 4.435109, 51.253444],
 ['P141', 4.4355804, 51.253825],
 ['P159', 4.4495068, 51.259815],
 ['P189', 4.391996958, 51.164014],
 ['P219', 4.4004772, 51.163794],
 ['P220', 4.391218, 51.18165],
 ['P239', 4.371189, 51.157605],
 ['P243', 4.383201147, 51.170694],
 ['P245', 4.3902351, 51.168411],
 ['P249', 4.4369445, 51.240375],
 ['P87', 4.3534352, 51.161472],
 ['P135', 4.4457976, 51.260159],
 ['P143', 4.4404288, 51.255159],
 ['P157', 4.4342995, 51.256377],
 ['P162', 4.3988934, 51.177503],
 ['P204', 4.4033597, 51.172804],
 ['P232', 4.4252168, 51.176284],
 ['P181', 4.3999749, 51.159779],
 ['P45', 4.3711708, 51.176821],
 ['P156', 4.441263, 51.244982],
 ['P258', 4.4395575, 51.199533],
 ['P261', 4.4283756, 

In [5]:
# make geojson files for each pair of coords
# for item in poi_coords:
   
#     isochrone = ors.isochrones(locations=[item], range=[600])
#     with open('./dummy/test_geo.geojson', 'w') as f:
#         json.dump(isochrone, f, ensure_ascii=False,indent=4)

In [6]:
# load geojson file into the program and check that it is a valid object
with open('./dummy/test_geo.geojson', 'r') as f:
    gj = geojson.load(f)

gj.is_valid

True

In [7]:
# reverse coords for use in folium
for item in poi_coords:
    item.reverse()

for item in poster_coords:
    item.reverse()

In [8]:
map = folium.Map(location=(poi_coords[0][:2]), tiles="cartodbpositron", min_zoom=11)

folium.Marker(poi_coords[0][:2]).add_to(map)

# for item in poster_coords:
    

folium.features.GeoJson(gj).add_to(map)

map

In [9]:
drivetime = Polygon(gj['features'][0]['geometry']['coordinates'][0])


count = 0
for lat, long, id in poster_coords:
    point = Point(long,lat)

    if point.within(drivetime) or point.touches(drivetime):
        folium.Marker(location= [lat, long],popup=item[2], icon= folium.Icon(color='pink', icon='star') ).add_to(map)
        count += 1
    else:
        folium.Marker(location= [lat, long],popup=item[2], icon= folium.Icon(color='cadetblue', icon='star') ).add_to(map)
print(count)

map


52


In [10]:

# returns area in square-degrees
drivetime.area

0.006820629596499976

In [23]:
# https://stackoverflow.com/questions/23697374/calculate-polygon-area-in-planar-units-e-g-square-meters-in-shapely

geod = Geod(ellps="WGS84")

area, perimeter = geod.geometry_area_perimeter(drivetime)

sqkm = area / 1000000

sqkm

52.991663173533325

In [22]:
density = count / abs(sqkm)

density

0.9812864304657527